# Cores & Memory: Streaming Kernels

## - Assignment 1 Miscellany
## - Review: Little's Law, CPUs vs. GPUs
## - Streaming Kernels
## - Arithmetic Intensity, Machine Balance, & the Roofline Model

## First-Week-Flops Miscellany

### My mistakes:

#### - It took a few tries to get the jupyter notebook job working, but it does now

#### - `nvcc` syntax: use `-lineinfo` for symbols in binaries; `-G` turns off all optimizations :(

#### - I'll try to include `module load cse6230` in future assignments.  (If it doesn't work, check if modules are loaded first)



### Useful commands:

#### - `pbsnodes -a` / `qnodes -a` on the head node

#### - `nvidia-smi`: present on all nodes with GPUs.  Useful in and of itself, but also good for detecting when no GPUs are present: `which nvidia-smi || echo "No GPUs"`

### Questions / confusion people have had:

#### - "Intel says this is a 6-core / 12-hardware thread package; PACE says this is a 12 core node.  Is PACE wrong?"

Try this on a compute node when you have X forwarding:

In [1]:
module load hwloc
lstopo

Machine (128GB total)
  NUMANode L#0 (P#0 64GB)
    Socket L#0 + L3 L#0 (15MB)
      L2 L#0 (256KB) + L1d L#0 (32KB) + L1i L#0 (32KB) + Core L#0 + PU L#0 (P#0)
      L2 L#1 (256KB) + L1d L#1 (32KB) + L1i L#1 (32KB) + Core L#1 + PU L#1 (P#1)
      L2 L#2 (256KB) + L1d L#2 (32KB) + L1i L#2 (32KB) + Core L#2 + PU L#2 (P#2)
      L2 L#3 (256KB) + L1d L#3 (32KB) + L1i L#3 (32KB) + Core L#3 + PU L#3 (P#3)
      L2 L#4 (256KB) + L1d L#4 (32KB) + L1i L#4 (32KB) + Core L#4 + PU L#4 (P#4)
      L2 L#5 (256KB) + L1d L#5 (32KB) + L1i L#5 (32KB) + Core L#5 + PU L#5 (P#5)
    HostBridge L#0
      PCIBridge
        PCI 15b3:1003
          Net L#0 "ib0"
          OpenFabrics L#1 "mlx4_0"
      PCIBridge
        PCI 10de:1023
      PCIBridge
        PCIBridge
          PCI 1a03:2000
      PCIBridge
        PCI 8086:1521
          Net L#2 "eth0"
        PCI 8086:1521
          Net L#3 "eth1"
      PCI 8086:8d02
        Block L#4 "sda"
  NUMANode L#1 (P#1 64GB)
    Socket L#1 + L3 L#1 (15MB)
      L2 L#6

You'll see that PACE has it right because, for all of the nodes we will be using, they have installed *two sockets per node*.

### There is now a grading script that you can try for yourself

`./grading-script.sh` will:

- split up your notebook into host and compute node components
- use your qsub expressions to run your compute node script on one of each type of node
- even though you tune for one type of node, it should run without crashing on any node

### For those seeking peak GPU performance, you can also control the "grid size" (number of thread blocks)

Add, e.g., `Gs=15` to the `run_fma_prof` and `run_fma_prof_opt` targets

This will let you experiment with ILP vs. TLP.

## Review: Little's Law

$\Huge L = \lambda W$

#### - $L$: amount of $x$ in a system.
#### - $\lambda$: arrival rate, $x$ / sec.
#### - $W$: time spent in the system (sec).

- Note that it has dimensions $x$, whatever it is we're trying to count.
- For pipelines, we often think of $\lambda$ as $x$ / cycle and $W$ as length of the pipeline in cycles.  Every unit but $x$ cancels out, so we can take whichever form is more convenient.

### Example from last time: how many independent FMAs are needed for peak flop/s?

#### - $W$: depth of the pipeline
#### - $\lambda$: arrival rate = # FPUs * # FMAs in a *vectorized* instruction

[[Intel's intrisics guide](https://software.intel.com/sites/landingpage/IntrinsicsGuide/)]

![vfmadd132ps](./images/intel-intrinsics.png)

latency = $W$, CPI (cycles per instruction) = 1 / $\lambda_v$, where $\lambda_v$ = throughput for *vectorized* FMAs.  Multiply by vector width (see operation pseudocode) to get full $\lambda$.

### Question from last time:

#### - What is the latency of FMA on the GPUs?  How could we estimate it?

### Quick Review: CPUs  (Hosts)

#### - One set of instructions per thread
#### - OS schedules threads (software multithreading), can *migrate* them between cores
#### - x86-64 (AVX2) instruction set has 32 256-bit vector registers per threads
#### - Parallelism via software multithread, hardware multithreading, superscalar execution, vectorization (**SIMD**)


### Quick Review: GPUs (Devices)

(See the nice illustrations from Prof. Vuduc's [slides](http://vuduc.org/cse6230/slides/cse6230-fa14--05-cuda.pdf), starting on slide 27)

#### - A *compute kernel* is a task that the host assigns to the device in a kernel launch

```c++

solveForX<<<ThreadsPerBlock,BlocksPerGrid>>>(A,x,y);
```

- Proceeds asynchronously from the host until the host requires the results

#### - The task is broken down into a **grid** of *independent* thread blocks

- The host has no control over which thread blocks are assigned where and in what order

#### - Each thread block is assigned to a streaming multiprocessor (SM), where it stays

- A SM may be assigned multiple thread blocks

#### - The SM breaks down the thread blocks into **warps** (32 threads): a warp shares an instruction set, so all (non-divergent) instructions are vectorized
#### - The SM issues instructions from multiple warps per cycle (sometimes multiple instructions per warp per cycle) 
#### - When a warp is stalled, another is scheduled
#### - Avoid: thread divergence


### Using `nvprof` to estimate the FMA latency

`nvprof` is a performance analysis tool like `perf` and `gprof` combined for the GPU

It can be inserted before a program in the same way as `perf`.  Here's the standard invocation of our
program from the first assignment.  First info about what kind of GPU I'm using:

In [7]:
nvidia-smi

Tue Aug 28 07:02:06 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.55                 Driver Version: 367.55                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  On   | 0000:81:00.0     Off |                    0 |
| N/A   24C    P0    25W / 250W |      0MiB / 16276MiB |      0%   E. Process |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [22]:
cd $CSE6230_DIR/assignments/first-week-flops
git checkout fma_loop_dev.cu
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000

nvcc -ccbin=icpc -Xcompiler '-fPIC'  -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_link.o
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
icpc -qopenmp -shared -Wl,-soname,libfma_cuda.so -o libfma_cuda.so fma_cuda_link.o fma_cuda.o fma_loop_dev.o -L/usr/local/pacerepov1/cuda/8.0.44/lib64 -Wl,-rpath,/usr/local/pacerepov1/cuda/8.0.44/lib64 -lcudart
icpc -qopenmp -o fma_prof fma_prof.o fma_omp.o fma_loop_host.o libfma_cuda.so -Wl,-rpath,.
OMP_PROC_BIND=spread OMP_NUM_THREADS=8  ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 1024
[./fma_prof]: 2.236700e-02 ela

Note that we are using the maximum thread block size (1024) and making two times as many thread blocks as there are SMs (56), leading to 2048 threads per SM, the maximum **occupancy**.

#### - **Occupancy**: the ratio of active warps in an SM (GPU) to the maximum number of warps per SM (GPU). 

A good measure of whether there is additional *thread-level parallelism* on the device.  Let's see if we can measure that.

(Note: for this GPU, I could get closer to 100% occupancy with a block size of 512 and twice as many blocks.  Why?)

In [21]:
nvprof --query-metrics | grep occupancy

              achieved_occupancy:  Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor
              achieved_occupancy:  Ratio of the average active warps per active cycle to the maximum number of warps supported on a multiprocessor


Let's try it out:

In [23]:
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000 PERF="nvprof --metrics achieved_occupancy"

OMP_PROC_BIND=spread OMP_NUM_THREADS=8 nvprof --metrics achieved_occupancy ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 1024
==3858== NVPROF is profiling process 3858, command: ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof]: 4.061890e-02 elapsed seconds
[./fma_prof]: 57344000000 flops executed
[./fma_prof]: 1.411757e+12 flop/s
==3858== Profiling application: ./fma_prof 0 114688 1024 112 250000 0.5 3.0
==3858== Profiling result:
==3858== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "Tesla P100-PCIE-16GB (0)"
    Kernel: __nv_static_56__43_tmpxft_00007909_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf
          1                        achieved_occupancy                        Achieved Occupancy    0.831391    0.831391    0.831391
    Kernel: fma_loop_dev(int, int, float*, float, float)
          1    

Okay, but let's look at the measure that we're actually interested, the percentag of peak flop/s

In [4]:
nvprof --query-metrics | grep flop | grep peak

              flop_hp_efficiency:  Ratio of achieved to peak half-precision floating-point operations
              flop_sp_efficiency:  Ratio of achieved to peak single-precision floating-point operations
              flop_dp_efficiency:  Ratio of achieved to peak double-precision floating-point operations
              flop_hp_efficiency:  Ratio of achieved to peak half-precision floating-point operations
              flop_sp_efficiency:  Ratio of achieved to peak single-precision floating-point operations
              flop_dp_efficiency:  Ratio of achieved to peak double-precision floating-point operations


In [25]:
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000 PERF="nvprof --metrics flop_sp_efficiency"

OMP_PROC_BIND=spread OMP_NUM_THREADS=8 nvprof --metrics flop_sp_efficiency ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 1024
==4217== NVPROF is profiling process 4217, command: ./fma_prof 0 114688 1024 112 250000 0.5 3.0
==4217== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==4217== Replaying kernel "__nv_static_56__43_tmpxft_00007909_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf" (done)           
==4217== Replaying kernel "fma_loop_dev(int, int, float*, float, float)" (done)           
[./fma_prof]: 1.543440e+00 elapsed seconds
[./fma_prof]: 57344000000 flops executed
[./fma_prof]: 3.715337e+10 flop/s
==4217== Profiling application: ./fma_prof 0 114688 1024 112 250000 0.5 3.0
==4217== Profiling result:
==4217== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Device "Tesla P100-P

So while we have good occupancy, we are acheiving only ~50% of peak flop/s.

First, let's unroll things just a little bit.

In [26]:
sed -i -e '/for (int j/i #pragma unroll 64' fma_loop_dev.cu
git diff fma_loop_dev.cu

diff --git a/assignments/first-week-flops/fma_loop_dev.cu b/assignments/first-week-flops/fma_loop_dev.cu
index c603504..d556288 100644
--- a/assignments/first-week-flops/fma_loop_dev.cu
+++ b/assignments/first-week-flops/fma_loop_dev.cu
@@ -9,6 +9,7 @@ fma_loop_dev (int N, int T, float *a, float b, float c)
   int num_threads = gridDim.x * blockDim.x;
 
   for (int i = my_thread; i < N; i+= num_threads) {
+#pragma unroll 64
     for (int j = 0; j < T; j++) {
       a[i] = a[i] * b + c;
     }


In [27]:
make run_fma_prof Nh=0 Bs=1024 Gs=$((56*2)) Nd=$((1024*56*2)) T=250000 PERF="nvprof --metrics flop_sp_efficiency"

nvcc -ccbin=icpc -Xcompiler '-fPIC'  -dc -o fma_loop_dev.o fma_loop_dev.cu
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
nvcc -ccbin=icpc -Xcompiler '-fPIC' -dlink  fma_cuda.o fma_loop_dev.o -o fma_cuda_link.o
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
icpc -qopenmp -shared -Wl,-soname,libfma_cuda.so -o libfma_cuda.so fma_cuda_link.o fma_cuda.o fma_loop_dev.o -L/usr/local/pacerepov1/cuda/8.0.44/lib64 -Wl,-rpath,/usr/local/pacerepov1/cuda/8.0.44/lib64 -lcudart
icpc -qopenmp -o fma_prof fma_prof.o fma_omp.o fma_loop_host.o libfma_cuda.so -Wl,-rpath,.
OMP_PROC_BIND=spread OMP_NUM_THREADS=8 nvprof --metrics flop_sp_efficiency ./fma_prof 0 114688 1024 112 250000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 114688, T = 250000, block size = 

Okay, now we're seeing about ~90% efficiency (thought it took a lot of unrolling!)

Now, according to the chart from last time:

![NVIDIA comparison](../processors/images/nvidia-table.png)

There are 64 single precision FPUs per SM on this Pascal GPU.  If I keep the same number of thread blocks, but set each block size to $64 / 2= 32$ (1 warp!), then there will be one thread per FPU.

Because each thread's computation as a loop dependency (iteration $i + 1$ cannot start until iteration $i$ has completed), we should only be issuing one FMA at the rate that an operation can traverse the pipeline.  Let's see:

In [30]:
make run_fma_prof Nh=0 Bs=32 Gs=$((56*2)) Nd=$((32*56*2)) T=8000000 PERF="nvprof --metrics flop_sp_efficiency --metrics achieved_occupancy"

OMP_PROC_BIND=spread OMP_NUM_THREADS=8 nvprof --metrics flop_sp_efficiency --metrics achieved_occupancy ./fma_prof 0 3584 32 112 8000000 0.5 3.0
[./fma_prof] Nh = 0, Nd = 3584, T = 8000000, block size = 32
==5138== NVPROF is profiling process 5138, command: ./fma_prof 0 3584 32 112 8000000 0.5 3.0
==5138== Some kernel(s) will be replayed on device 0 in order to collect all events/metrics.
==5138== Replaying kernel "__nv_static_56__43_tmpxft_00007909_00000000_7_fma_cuda_cpp1_ii_3fd49640__Z14fma_initializeiPf" (done)           
==5138== Replaying kernel "fma_loop_dev(int, int, float*, float, float)" (done)           
[./fma_prof]: 9.380638e+00 elapsed seconds
[./fma_prof]: 57344000000 flops executed
[./fma_prof]: 6.113017e+09 flop/s
==5138== Profiling application: ./fma_prof 0 3584 32 112 8000000 0.5 3.0
==5138== Profiling result:
==5138== Metric result:
Invocations                               Metric Name                        Metric Description         Min         Max         Avg
Dev

Note that our achieved occupancy is ~1/32 = 32 / 1024, which makes sense.  Our efficiency is ~16%, so our estimated pipeline depth is **$1 / 0.16 \approx 6$**.

You could repeat this for the Kepler GPUs if you wanted, but the number would be quite different.
As the chart above shows, the design of NVIDIA GPUs is not stable: Kepler has a few large SMs; Pascal has a lot of smaller SMs; Maxwell (not available on pace-ice) is in the middle, but almost completely abandons double precision arithmetic, which makes me sad :(.